In [1]:
from elastic_db.elasticsearch import elastic_conection, NLPmodelIndex

es = elastic_conection()
index = NLPmodelIndex(es=es, workspace_id="dc1e7b3d-9137-4a20-a99c-d0d2029ef170-cedae")
index.index_name

'nlp_model-dc1e7b3d-9137-4a20-a99c-d0d2029ef170-cedae-02022021092147'

In [2]:
query = {
  "size": 10000,
  "_source": ["intent", "examples.text"], 
  "query": {
    "bool": {
      "must": [
        {"term": {
          "doc_type.keyword": {
            "value": "intent"
          }
        }}
      ]
    }
  }
}

intents = es.search(index=index.index_name, body=query)
intents = [intent["_source"] for intent in intents["hits"]["hits"]]
intents[:1]

[{'examples': [{'text': 'Como consigo pagar minha conta atrasada?'},
   {'text': 'Como tirar uma nova conta deste mês?'},
   {'text': 'como vejo o boleto que paguei ontem?'},
   {'text': 'Desejo imprimir a conta a ser paga'},
   {'text': 'Não conseguimos a 2a via'},
   {'text': 'necessito do codigo de barra para pagar um debito'},
   {'text': 'passei do dia de pagamento não consigo mais pagar minha conta de água'},
   {'text': 'poderia me ajudar a pagar uma conta atrasada?'},
   {'text': 'preciso imprimir segunda via de boleto'},
   {'text': 'quero gerar segunda via de pagamento'},
   {'text': 'Quero minha 2 via'},
   {'text': 'Quero pagar minha conta atrasada'},
   {'text': 'Quero tirar uma segunda via'},
   {'text': 'seria possível eu pagar uma conta atrasada?'}],
  'intent': 'Emitir_SegundaViaConta'}]

In [4]:
import pandas as pd

import json

file = open('/home/cloves/Downloads/skill-IBM_CEDAE_PRD_PTBR.json', "r") 
workspace = json.load(file)
workspace["intents"][-1]

{'intent': 'Trocar_TitularidadeConta',
 'examples': [{'text': 'alteração de titularidade'},
  {'text': 'aluguei um imóvel e preciso trocar a titularidade da conta de água'},
  {'text': 'como faço para por minha conta de água no nome de outra pessoa?'},
  {'text': 'como faço para trocar a titularidade da conta de água do meu estabelecimento?'},
  {'text': 'como mudar o nome da conta?'},
  {'text': 'preciso trocar a titularidade da conta'},
  {'text': 'quero trocar a titularidade da minha conta de água'}],
 'description': 'Quando cliente CEDAE desejar alterar a titularidade da conta.'}

In [5]:
intents_name = []
examples_text = []
intents_8 = []
for intent in workspace["intents"]:
    intent_name = intent["intent"]
    if len(intent["examples"]) >= 8:
        intents_8.append(intent)
        for example in intent["examples"]:
            intents_name.append(intent_name)
            examples_text.append(example["text"].replace(",", ""))
        
print(len(intents_name), len(examples_text))

327 327


In [15]:
# def kfolders(lst, k):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), k):
#         yield lst[i:i + k]

In [6]:
import math
import random

random.seed(42)

def split_train_test(examples_text, intents_name, percentage=0.375):
    qt = math.ceil(len(examples_text)*percentage) 
    random.Random(42).shuffle(examples_text)
    random.Random(42).shuffle(intents_name)
    all_indexs = set(range(0, len(examples_text)))
    test_indexs = set(random.Random(42).sample(all_indexs, qt))
    train_indexs = all_indexs - test_indexs
    test_examples_text = [examples_text[index] for index in test_indexs]
    test_intents_name = [intents_name[index] for index in test_indexs]
    train_examples_text = [examples_text[index] for index in train_indexs]
    train_intents_name = [intents_name[index] for index in train_indexs]
    return train_examples_text, train_intents_name, \
           test_examples_text, test_intents_name

def split_train_test_per_intent(intents, percentage=0.3):
    test_examples_text = []
    test_intents_name = []
    train_examples_text = []
    train_intents_name = []
    for intent in intents:
        qt = math.ceil(len(intent["examples"])*percentage)
        all_indexs = set(range(0, len(intent["examples"])))
        test_indexs = set(random.Random(42).sample(all_indexs, qt))
        train_indexs = all_indexs - test_indexs
        test_examples_text += [intent["examples"][index]["text"].replace(",", "") for index in test_indexs]
        test_intents_name += [intent["intent"]]*len(test_indexs)
        train_examples_text += [intent["examples"][index]["text"].replace(",", "") for index in train_indexs]
        train_intents_name += [intent["intent"]]*len(train_indexs)
    random.Random(42).shuffle(train_examples_text), \
    random.Random(42).shuffle(train_intents_name), \
    random.Random(42).shuffle(test_examples_text), \
    random.Random(42).shuffle(test_intents_name)
    return train_examples_text, \
           train_intents_name, \
           test_examples_text, \
           test_intents_name

In [42]:
# X_train, y_train, X_test, y_test = split_train_test(examples_text, intents_name)

In [7]:
X_train, y_train, X_test, y_test = split_train_test_per_intent(intents_8)

In [6]:
print(len(X_train), len(y_train))

221 221


In [7]:
print(len(X_test), len(y_test))

106 106


In [8]:
import pandas as pd

test_data = [{"example": example, "intent": intent} for example, intent in zip(X_test, y_test)]
df_test = pd.DataFrame(data=test_data)
df_test.head()

,example,intent
0,quero ir numa agência da CEDAE,Localizar_AgenciasCEDAE
1,Boa Noite,General_Greetings
2,quero um relatório dos meus pagamentos,Consultar_ContasPagas
3,você sabe falar sobre o que,Bot_Capabilities
4,cobranca indevida,Revisar_Conta


In [9]:
df_test.to_csv("data/test_data_cedae.csv", sep=",", index=False)

In [10]:
train_data = [{"example": example, "intent": intent} for example, intent in zip(X_train, y_train)]
df_train = pd.DataFrame(data=train_data)
df_train.head()

,example,intent
0,A água está barrenta,Reportar_QualidadeDeAgua
1,quero meu extrato de pagamento dos últimos meses,Consultar_ContasPagas
2,Contaminação da agua,Reportar_QualidadeDeAgua
3,Estou conversando com um robô?,Bot_About
4,Quero consultar meu CPF,Consultar_MatriculaCPF


In [11]:
df_train.to_csv("data/train_data_cedae.csv", sep=",", index=False)

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("data/train_data_cedae.csv")
df.head()

,example,intent
0,A água está barrenta,Reportar_QualidadeDeAgua
1,quero meu extrato de pagamento dos últimos meses,Consultar_ContasPagas
2,Contaminação da agua,Reportar_QualidadeDeAgua
3,Estou conversando com um robô?,Bot_About
4,Quero consultar meu CPF,Consultar_MatriculaCPF


In [10]:
df.intent.nunique()

20